In [1]:
!pip install datasets transformers

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import pickle as pkl
from sklearn.metrics import classification_report, confusion_matrix

from functools import partial

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import nltk

import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

In [4]:
file_names = ["/content/discrimination.csv", "/content/hci_harms_df.csv", "/content/malicious_activity.csv", "/content/misinfo.csv"]

In [5]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
#model = torch.nn.DataParallel(model)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [9]:
for file in file_names:
  model.eval()
  model.to(device)
  df = pd.read_csv(file)
  text = df['text'].tolist()
  labels = df['binary_label'].tolist()
  data_loader = DataLoader(text, batch_size=1, shuffle=False)

  activations = []
  for batch in data_loader:
      encoded_text = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
      output = model(**encoded_text)
      activations.extend(output.pooler_output.detach().cpu().numpy())

  results = {}
  results['activations'] = activations
  results['labels'] = labels

  domain = file.split("/")[-1]
  domain = domain.split(".")[0]
  pkl.dump(results, open(f"/content/bert_{domain}.pkl", "wb"))
  print(f"completed file: {file}")

completed file: /content/hci_harms_df.csv
